<a href="https://colab.research.google.com/github/wcaine93/DS-ML-Project/blob/main/data-cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code

In [111]:
# imports
import pandas as pd
import numpy as np

In [112]:
# place data in dataframe
df = pd.read_csv("data/Project_Data.csv")

In [113]:
#User defined function for dropping  rows that contain outliers

def drop_outliers(df, column):
    # I've changed the code to use mean and standard deviation method
    mean = df[column].mean()
    stdev = df[column].std()

    lower_bound = mean - 3 * stdev  # Lower bound
    upper_bound = mean + 3 * stdev  # Upper bound

    # Filter and keep only non-outlier rows
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [114]:
#User defined function for substituting outliers with the mean values that do not contain the outliers
def replace_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)

    # Compute mean without outliers

#The .loc[] function in pandas is used for label-based indexing to access specific rows and columns in a DataFrame. It allows for filtering, selecting, and updating data based on labels or boolean conditions. The sign ~ means excluding something. ~outliers means that rows excluding outliers


    mean_without_outliers = df.loc[~outliers, column].mean()

    # Replace outliers with the mean
    df.loc[outliers, column] = mean_without_outliers

    return df

## Clean the Data

#### Duplicate and Negative Values

In the data inspection, we found that there were no negative values or duplicate records.

#### Missing Values

I will remove the records that are missing the `Gender` feature or `Overdrawn` label.

I will replace the missing `Age` with the average `Age` (19).

In [115]:
# drop data missing Gender or Overdrawn
df.dropna(subset=['Gender', 'Overdrawn'], inplace=True)
print(df.shape)

(446, 5)


4 records were removed, which is consistent with what we found during inspection.

In [116]:
# convert missing Age data to mean
df['Age'] = df['Age'].fillna(df['Age'].mean())
print(df.isnull().sum())

Unnamed: 0    0
Age           0
Gender        0
DaysDrink     0
Overdrawn     0
dtype: int64


No missing data remains.

### Outliers

#### Age

Because we found that the `Age` outliers identified the standard deviations method wouldn't integrate well with the data, I will drop these from the data set.

The remaining `Age` outliers, identified using the quantiles method, I will replace with the average value (19).

In [117]:
# drop outliers using the standard deviations method
df = drop_outliers(df, 'Age')
print(df.shape)

(435, 5)


11 records were removed, which is consistent with what we found during inspection.

In [118]:
# replace quantiles outliers with the average Age
df = replace_outliers(df, 'Age')

#### DaysDrink

Because of the high kurtosis of `DaysDrink` data we found during inspection, I will bin the `DaysDrink` column instead of removing the outliers (see [Data Type Cast](#scrollTo=kijWflTjlcwS&line=1&uniqifier=1)).

### Data Type Cast

#### DaysDrink

In [119]:
bin1 = df[df['DaysDrink'] == 0]
print(len(bin1))
bin2 = df[(df['DaysDrink'] >= 1) & (df['DaysDrink'] <= 4)]
print(len(bin2))
bin3 = df[(df['DaysDrink'] >= 5) & (df['DaysDrink'] <= 10)]
print(len(bin3))
bin4 = df[(df['DaysDrink'] >= 11) & (df['DaysDrink'] <= 30)]
print(len(bin4))

126
113
93
103


I have chosen the above bins because they roughly evenly distribute the data:

\\

Does not drink ("None"): `DaysDrink == 0`

Barely drinks ("Low"): `1 <= DaysDrink <= 4`

Drinks somewhat ("Moderate"): `5 <= DaysDrink <= 10`

Drnks a lot ("High"): `11 <= DaysDrink`


In [120]:
# cut DaysDrink into chosen bins
bins = [-1, 0, 4, 10, 999]
labels = ['None', 'Low', 'Moderate', 'High']
df['DaysDrink'] = pd.cut(df['DaysDrink'], bins=bins, labels=labels)

<ipython-input-120-406899a988fa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DaysDrink'] = pd.cut(df['DaysDrink'], bins=bins, labels=labels)


#### Gender

We will remove inconsistent values in `Gender` and create binary columns for if a record corresponds to a male or a female.

In [121]:
# make Gender data consistent (M -> Male, F -> Female)
df['Gender'] = df['Gender'].map({"Male": "Male", "M": "Male", "Female": "Female", "F": "Female"})
print(df['Gender'].unique())

['Female' 'Male']


<ipython-input-121-f3206ff6dc70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gender'] = df['Gender'].map({"Male": "Male", "M": "Male", "Female": "Female", "F": "Female"})


In [122]:
# create columns for numeric Gender data (and order columns for final format)
df = pd.DataFrame(
    {
        'Gender': df['Gender'],
        'Gender_Male': df['Gender'].map({"Male": 1, "Female": 0}),
        'Gender_Female': df['Gender'].map({"Male": 0, "Female": 1}),
        'Age': df['Age'],
        'DaysDrink': df['DaysDrink'],
        'Overdrawn': df['Overdrawn']
    }
    )
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 0 to 449
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Gender         435 non-null    object  
 1   Gender_Male    435 non-null    int64   
 2   Gender_Female  435 non-null    int64   
 3   Age            435 non-null    float64 
 4   DaysDrink      435 non-null    category
 5   Overdrawn      435 non-null    float64 
dtypes: category(1), float64(2), int64(2), object(1)
memory usage: 21.0+ KB
None


#### Age

In [123]:
df['Age'] = df['Age'].astype('int64')

#### Overdrawn

In [124]:
df['Overdrawn'] = df['Overdrawn'].astype('int64')

## Final Data

In [126]:
print(df.shape)
print(df.info())

(435, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 0 to 449
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Gender         435 non-null    object  
 1   Gender_Male    435 non-null    int64   
 2   Gender_Female  435 non-null    int64   
 3   Age            435 non-null    int64   
 4   DaysDrink      435 non-null    category
 5   Overdrawn      435 non-null    int64   
dtypes: category(1), int64(4), object(1)
memory usage: 21.0+ KB
None


In [128]:
df.to_csv('data/clean_data.csv')

## Reference for decisions

Kang H. (2013). The prevention and handling of the missing data. *Korean journal of anesthesiology, 64(5), 402–406*. https://doi.org/10.4097/kjae.2013.64.5.402